In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import j1  # Bessel function of the first kind
from ipywidgets import interact, widgets
import pandas as pd

In [2]:
# Constants
rho0 = 1
c = 1500
r = 10

In [3]:
# Create sliders for frequency and transducer radius
f_slider = widgets.IntSlider(
    min=1, max=50, step=1, value=1, description='Frequency (kHz)',
    style={'description_width': 'initial'}
)
a_slider = widgets.FloatSlider(
    min=0.1, max=1, step=0.05, value=0.5, description='Aperture Radius (m)',
    style={'description_width': 'initial'}
)

In [4]:
# Create widgets for plot type and scale
plot_type_widget = widgets.Dropdown(
    options=['Cartesian', 'Polar'],
    value='Polar',
    description='Plot Type:'
)

plot_scale_widget = widgets.Dropdown(
    options=['Decibels', 'Linear'],
    value='Decibels',
    description='Scale:'
)

In [5]:
def compute_far_field_intensity(f, a):
    omega = 2 * np.pi * f * 1000
    k = omega / c
    theta = np.arange(-90, 90.1, 0.1)
    u = k * a * np.sin(theta * np.pi / 180)

    # Projection Intensity calculation
    projInt = np.abs((rho0 * c * k * a * a / (2 * r)) * 
                     (2 * j1(u) / u)) ** 2

    # Handle division by zero at theta = 0
    ind = np.where(theta == 0)[0]
    axisInt = np.abs(rho0 * c * k * a * a / (2 * r)) ** 2
    projInt[ind] = axisInt

    return projInt, theta, axisInt

In [20]:
def update_far_field_intensity_cartesian_polar_plot(f, a, plotType, plotScale):
    # Calculate far field intensity
    projInt, theta, axisInt = compute_far_field_intensity(f, a)

    plt.close('all')  # Close any existing plots

    if plotType == 'Cartesian':
        plt.figure(figsize=(6, 4))
        if plotScale == "Decibels":
            plt.plot(theta, 10 * np.log10(projInt / axisInt), linewidth=2)
            plt.ylim([-40, 0])
            plt.xlim([-90, 90])
            plt.xlabel('Angle (degrees)', fontsize=14)
            plt.ylabel('Beam Level (dB)', fontsize=14)
            plt.grid(True)
        else:
            plt.plot(theta, projInt / axisInt, linewidth=2)
            plt.ylim(10 ** (np.array([-40, 0]) / 10))
            plt.xlim([-90, 90])
            plt.xlabel('Angle (degrees)', fontsize=14)
            plt.ylabel('Beam Level', fontsize=14)
            plt.grid(True)
    
    elif plotType == 'Polar':
        plt.figure(figsize=(6, 6))
        if plotScale == "Decibels":
            plt.polar(theta * np.pi / 180, 10 * np.log10(projInt / axisInt), linewidth=2)
            ax = plt.gca()
            ax.set_ylim([-40, 0])
            ax.set_theta_zero_location('N')  # 'top'
            ax.set_thetalim([np.radians(-90), np.radians(90)])
            ax.set_title('Beam Pattern (dB)', fontsize=14)
        else:
            plt.polar(theta * np.pi / 180, projInt / axisInt, linewidth=2)
            ax = plt.gca()
            ax.set_ylim(10 ** (np.array([-40, 0]) / 10))
            ax.set_theta_zero_location('N')  # 'top'
            ax.set_thetalim([np.radians(-90), np.radians(90)])
            ax.set_title('Beam Pattern (Linear Scale)', fontsize=14)

    # Compute beam widths
    ind3dB = np.where(10 * np.log10(projInt / axisInt) >= -3)[0]
    ind6dB = np.where(10 * np.log10(projInt / axisInt) >= -6)[0]
    ind10dB = np.where(10 * np.log10(projInt / axisInt) >= -10)[0]

    bw3dB = np.abs(theta[ind3dB[0]])
    bw6dB = np.abs(theta[ind6dB[0]])
    bw10dB = np.abs(theta[ind10dB[0]])

    # Define labels and values
    bwLabels = ['down 3 dB', 'down 6 dB', 'down 10 dB']
    bwValues = [bw3dB, bw6dB, bw10dB]

    # Create a DataFrame
    df = pd.DataFrame({
        'Beam Width (degrees)': bwValues,
        'Definition': bwLabels
    })

    # Print the DataFrame
    print(df)

    # Show fig
    plt.show()


In [21]:
def far_field_intensity_x_y_plane_calculation(f, a, r, cr):
    omega = 2 * np.pi * f * 1000
    k = omega / c

    # Calculations
    SL_dB = 180
    SL_lin = 10**(SL_dB / 20)
    U0 = SL_lin / (rho0 * c * k * a * a / 2)

    x = np.linspace(-cr, cr, 150)
    y = np.linspace(0, r, 100)
    X, Y = np.meshgrid(x, y)

    thetaXY = np.arctan2(X, Y)
    rXY = np.sqrt(X**2 + Y**2)
    u1 = k * a * np.sin(thetaXY)
    u1[u1 == 0] = np.nan  # Avoid division by zero

    projIntXY = np.abs((rho0 * c * k * U0 * a * a / (2 * rXY)) *
                    (2 * j1(u1) / u1))**2

    return x, y, SL_dB, projIntXY

In [22]:
def update_far_field_intensity_x_y_plane_plot(f, a, plotScale):
    # Calculate far field intensity on x-y plane
    x, y, SL_dB, projIntXY = far_field_intensity_x_y_plane_calculation(f, a, r=10, cr=10)

    # Close any existing plots
    plt.close('all')

    # Plotting
    plt.figure(1)
    if plotScale == 'Decibels':
        plt.imshow(10 * np.log10(projIntXY), extent=(x.min(), x.max(), y.min(), y.max()), origin='lower')
        plt.colorbar(label='Intensity (dB re 1μPa)')
        plt.clim(SL_dB - 90, SL_dB)
    else:
        plt.imshow(projIntXY / 1e6, extent=(x.min(), x.max(), y.min(), y.max()), origin='lower')
        plt.colorbar(label='Intensity (μPa²)')
        plt.clim(10**((SL_dB - 90) / 10), SL_dB)
        
    plt.xlabel('Cross Range (m)')
    plt.ylabel('Range (m)')
    plt.gca().set_aspect('auto')
    plt.show()

In [23]:
# Explore far field intensity across angles
_ = interact(
    update_far_field_intensity_cartesian_polar_plot,
    f=f_slider,
    a=a_slider,
    plotType=plot_type_widget,
    plotScale=plot_scale_widget,
)

interactive(children=(IntSlider(value=18, description='Frequency (kHz)', max=50, min=1, style=SliderStyle(desc…

In [24]:
# Explore far field intensity in x-y plane
_ = interact(
    update_far_field_intensity_x_y_plane_plot,
    f=f_slider,
    a=a_slider,
    plotScale=plot_scale_widget,
)

interactive(children=(IntSlider(value=18, description='Frequency (kHz)', max=50, min=1, style=SliderStyle(desc…